# Reddit Marketing AI Agent - Complete Example Workflow (Direct Functions)

This notebook demonstrates the complete workflow of the Reddit Marketing AI Agent using direct function calls instead of API endpoints.

## Features Demonstrated:
1. **Setup & Configuration** - Environment validation and service initialization
2. **Organization Setup** - Create and configure an organization
3. **Document Ingestion** - Multiple methods (direct content, file upload, URL scraping)
4. **Campaign Creation** - Create and configure a marketing campaign
5. **Topic Discovery** - Extract relevant topics from documents
6. **Subreddit Discovery** - Find relevant subreddits based on topics
7. **Post Discovery** - Find relevant posts in target subreddits
8. **Response Generation** - AI-generated contextual responses
9. **Response Execution** - Post responses to Reddit (with safety controls)
10. **Analytics & Reporting** - Comprehensive performance analysis

## Safety Features:
- **Reddit Posting Control**: `ACTUALLY_POST_TO_REDDIT = False` prevents accidental posting
- **Credential Validation**: Checks for required API keys
- **Error Handling**: Graceful handling of service failures
- **Independent Cells**: Each step can be run independently

## 1. Setup & Configuration

In [5]:
import os
import sys
from typing import  Any
from datetime import datetime
import time

# Add the app directory to Python path
sys.path.append('app')

# Import all necessary services and managers
from app.services.document_service import DocumentService
from app.services.campaign_service import CampaignService
from app.services.reddit_service import RedditService
from app.services.llm_service import LLMService
from app.services.analytics_service import AnalyticsService
from app.services.scraper_service import WebScraperService

from app.managers.document_manager import DocumentManager
from app.managers.campaign_manager import CampaignManager
from app.managers.analytics_manager import AnalyticsManager

from app.storage.json_storage import JsonStorage
from app.storage.vector_storage import VectorStorage

from app.clients.llm_client import LLMClient
from app.clients.reddit_client import RedditClient
from app.clients.storage_client import VectorStorageClient

from app.models.campaign import (
    CampaignCreateRequest, SubredditDiscoveryRequest, SubredditDiscoveryByTopicsRequest,
    PostDiscoveryRequest, ResponseGenerationRequest, ResponseExecutionRequest,
    ResponseTone
)
from app.models.document import DocumentCreateRequest, DocumentIngestURLRequest, DocumentQuery

from app.core.settings import settings

# Configuration
ORGANIZATION_ID = "example-org-2024"
ORGANIZATION_NAME = "Example Organization"

# Safety control - Set to True only when you want to actually post to Reddit
ACTUALLY_POST_TO_REDDIT = True

# Reddit credentials (replace with your actual credentials)
REDDIT_CREDENTIALS = {
     "client_id": os.getenv("REDDIT_CLIENT_ID"),
    "client_secret": os.getenv("REDDIT_CLIENT_SECRET"),
    "username": os.getenv("REDDIT_USERNAME"),
    "password": os.getenv("REDDIT_PASSWORD")
}

print("🚀 Reddit Marketing AI Agent - Example Workflow (Direct Functions)")
print(f"📅 Started at: {datetime.now()}")
print(f"🏢 Organization: {ORGANIZATION_NAME} ({ORGANIZATION_ID})")
print(f"⚠️  Reddit Posting: {'ENABLED' if ACTUALLY_POST_TO_REDDIT else 'DISABLED (Safe Mode)'}")
print("\n" + "="*60)

🚀 Reddit Marketing AI Agent - Example Workflow (Direct Functions)
📅 Started at: 2025-06-21 07:43:58.493015
🏢 Organization: Example Organization (example-org-2024)
⚠️  Reddit Posting: ENABLED



In [6]:
# Initialize all services and managers
print("🔧 Initializing Services and Managers...")

# Storage layer
json_storage = JsonStorage()
vector_storage_client = VectorStorageClient()
vector_storage = VectorStorage(vector_storage_client)

# Managers
document_manager = DocumentManager(json_storage)
campaign_manager = CampaignManager(json_storage)
analytics_manager = AnalyticsManager(campaign_manager, document_manager)

# Clients
llm_client = LLMClient()
reddit_client = RedditClient(
    client_id=REDDIT_CREDENTIALS["client_id"],
    client_secret=REDDIT_CREDENTIALS["client_secret"],
    username=REDDIT_CREDENTIALS.get("username"),
    password=REDDIT_CREDENTIALS.get("password")
)
web_scraper_service = WebScraperService()

# Services
llm_service = LLMService(llm_client)
reddit_service = RedditService(json_storage, reddit_client)
document_service = DocumentService(document_manager, vector_storage, web_scraper_service)
campaign_service = CampaignService(campaign_manager, document_service, reddit_service, llm_service)
analytics_service = AnalyticsService(analytics_manager)

print("✅ All services initialized successfully!")

# Helper functions
def print_result(title: str, success: bool, message: str, data: Any = None):
    """Pretty print service results."""
    print(f"\n📋 {title}")
    print("-" * 40)
    
    status = "✅" if success else "❌"
    print(f"{status} Status: {message}")
    
    if data:
        if isinstance(data, dict):
            for key, value in data.items():
                print(f"📊 {key}: {value}")
        elif isinstance(data, list):
            print(f"📊 Items: {len(data)}")
            for i, item in enumerate(data[:3], 1):  # Show first 3 items
                print(f"   {i}. {str(item)[:80]}...")
        else:
            print(f"📊 Data: {str(data)[:100]}...")

print("✅ Helper functions loaded")

🔧 Initializing Services and Managers...
✅ All services initialized successfully!
✅ Helper functions loaded


In [7]:
# Check environment and settings
print("🔍 Environment Check")
print("-" * 40)

required_keys = {
    "OPENAI_API_KEY": settings.OPENAI_API_KEY,
    "GOOGLE_API_KEY": settings.GOOGLE_API_KEY
}

optional_keys = {
    "GROQ_API_KEY": settings.GROQ_API_KEY,
    "FIRECRAWL_API_KEY": settings.FIRECRAWL_API_KEY,
    "LANGCHAIN_PROJECT": settings.LANGCHAIN_PROJECT
}

print("Required API Keys:")
for key, value in required_keys.items():
    status = "✅" if value else "❌"
    print(f"   {status} {key}: {'Set' if value else 'Missing'}")

print("\nOptional API Keys:")
for key, value in optional_keys.items():
    status = "✅" if value else "⚠️"
    print(f"   {status} {key}: {'Set' if value else 'Not set'}")

print(f"\n📁 Data Directory: {settings.DATA_DIR}")
print(f"🤖 Default Model: {settings.MODEL_NAME}")
print(f"🔍 Embedding Provider: {settings.EMBEDDING_PROVIDER}")

🔍 Environment Check
----------------------------------------
Required API Keys:
   ✅ OPENAI_API_KEY: Set
   ✅ GOOGLE_API_KEY: Set

Optional API Keys:
   ✅ GROQ_API_KEY: Set
   ✅ FIRECRAWL_API_KEY: Set
   ⚠️ LANGCHAIN_PROJECT: Not set

📁 Data Directory: data
🤖 Default Model: gpt-4o
🔍 Embedding Provider: openai


## 2. Organization Setup

In [8]:
# Get or create organization
print("🏢 Setting up Organization")

organization = document_service.get_or_create_organization(ORGANIZATION_ID, ORGANIZATION_NAME)

print(f"✅ Organization: {organization.name}")
print(f"📊 ID: {organization.id}")
print(f"📄 Documents: {organization.documents_count}")
print(f"📅 Created: {organization.created_at}")
print(f"🔄 Active: {organization.is_active}")

# List all organizations
all_organizations = document_service.list_organizations()
print(f"\n📋 Total organizations in system: {len(all_organizations)}")
for org in all_organizations:
    print(f"   - {org.name} ({org.id}): {org.documents_count} documents")

🏢 Setting up Organization
✅ Organization: Example Organization
📊 ID: example-org-2024
📄 Documents: 0
📅 Created: 2025-06-21 02:14:13.899903+00:00
🔄 Active: True

📋 Total organizations in system: 1
   - Example Organization (example-org-2024): 0 documents


## 3. Document Ingestion

We'll demonstrate all three document ingestion methods:
1. **Direct Content Input** - Paste content directly
2. **Simulated File Upload** - Simulate uploading a file
3. **URL Scraping** - Scrape content from a URL

In [9]:
# Method 1: Direct Content Input
print("📄 Method 1: Direct Content Input")

direct_documents = [
    {
        "title": "Python Best Practices Guide",
        "content": """
        Python Best Practices for Clean Code
        
        Writing clean, maintainable Python code is essential for any developer. Here are some key best practices:
        
        1. Follow PEP 8 Style Guide
        - Use 4 spaces for indentation
        - Keep lines under 79 characters
        - Use descriptive variable names
        
        2. Write Docstrings
        - Document all functions and classes
        - Use triple quotes for docstrings
        - Follow Google or NumPy docstring conventions
        
        3. Use Type Hints
        - Add type hints to function parameters and return values
        - Use typing module for complex types
        - Helps with IDE support and code documentation
        
        4. Error Handling
        - Use specific exception types
        - Handle exceptions gracefully
        - Log errors appropriately
        
        5. Testing
        - Write unit tests for all functions
        - Use pytest for testing framework
        - Aim for high test coverage
        
        These practices will help you write more maintainable and professional Python code.
        """,
        "metadata": {
            "category": "programming",
            "language": "python",
            "difficulty": "intermediate"
        }
    },
    {
        "title": "Machine Learning Fundamentals",
        "content": """
        Introduction to Machine Learning
        
        Machine Learning (ML) is a subset of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed.
        
        Types of Machine Learning:
        
        1. Supervised Learning
        - Uses labeled training data
        - Examples: Classification, Regression
        - Algorithms: Linear Regression, Decision Trees, Random Forest
        
        2. Unsupervised Learning
        - Works with unlabeled data
        - Examples: Clustering, Dimensionality Reduction
        - Algorithms: K-Means, PCA, DBSCAN
        
        3. Reinforcement Learning
        - Learns through interaction with environment
        - Uses rewards and penalties
        - Examples: Game playing, Robotics
        
        Key Concepts:
        - Feature Engineering: Selecting and transforming input variables
        - Model Training: Teaching the algorithm using training data
        - Model Evaluation: Testing performance on unseen data
        - Overfitting: When model performs well on training but poorly on new data
        
        Popular Python Libraries:
        - Scikit-learn: General-purpose ML library
        - TensorFlow: Deep learning framework
        - PyTorch: Research-focused deep learning
        - Pandas: Data manipulation and analysis
        - NumPy: Numerical computing
        """,
        "metadata": {
            "category": "machine-learning",
            "difficulty": "beginner",
            "topics": ["supervised", "unsupervised", "reinforcement"]
        }
    }
]

# Ingest direct content documents
success, message, direct_doc_ids = document_service.ingest_documents(
    documents=direct_documents,
    org_id=ORGANIZATION_ID,
    org_name=ORGANIZATION_NAME
)

print_result("Direct Content Ingestion", success, message, {
    "document_ids": direct_doc_ids,
    "documents_ingested": len(direct_doc_ids) if direct_doc_ids else 0
})

print(f"\n📝 Stored {len(direct_doc_ids) if direct_doc_ids else 0} document IDs from direct content")

📄 Method 1: Direct Content Input


Calculating embeddings: 1it [00:01,  1.21s/it]
Calculating embeddings: 1it [00:00,  1.08it/s]
Document e2b78f2d-ec18-40a6-9ffd-9eaf861d1734_chunk_0 contains `meta` values of unsupported types for the keys: topics. These items will be discarded. Supported types are: str, int, float, bool.



📋 Direct Content Ingestion
----------------------------------------
✅ Status: Successfully ingested 2 documents (2 chunks)
📊 document_ids: ['df433591-c4ff-499b-84ea-4d9c087a9fc2', 'e2b78f2d-ec18-40a6-9ffd-9eaf861d1734']
📊 documents_ingested: 2

📝 Stored 2 document IDs from direct content


In [10]:
# Method 2: Simulated File Upload
print("\n📁 Method 2: Simulated File Upload")

# Simulate file content (in real scenario, this would be read from an uploaded file)
file_content = """
Web Development with Python and FastAPI

FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints.

Key Features:
- Fast: Very high performance, on par with NodeJS and Go
- Fast to code: Increase the speed to develop features by about 200% to 300%
- Fewer bugs: Reduce about 40% of human (developer) induced errors
- Intuitive: Great editor support with completion everywhere
- Easy: Designed to be easy to use and learn
- Short: Minimize code duplication
- Robust: Get production-ready code with automatic interactive documentation

Getting Started:

1. Installation
```bash
pip install fastapi uvicorn
```

2. Basic Example
```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}
```

3. Run the server
```bash
uvicorn main:app --reload
```

Advanced Features:
- Automatic API documentation with Swagger UI
- Data validation using Pydantic models
- Dependency injection system
- Background tasks
- WebSocket support
- Authentication and authorization
- Database integration

FastAPI is perfect for building modern web APIs and microservices.
"""

# Create document from "file" content
file_documents = [{
    "title": "FastAPI Web Development Guide",
    "content": file_content,
    "metadata": {
        "source": "simulated_file_upload",
        "filename": "fastapi_guide.txt",
        "category": "web-development",
        "framework": "fastapi"
    }
}]

# Ingest file content
success, message, file_doc_ids = document_service.ingest_documents(
    documents=file_documents,
    org_id=ORGANIZATION_ID
)

print_result("File Content Ingestion", success, message, {
    "document_ids": file_doc_ids,
    "documents_ingested": len(file_doc_ids) if file_doc_ids else 0
})

print(f"\n📝 Stored {len(file_doc_ids) if file_doc_ids else 0} document IDs from file content")


📁 Method 2: Simulated File Upload


Calculating embeddings: 1it [00:00,  1.30it/s]


📋 File Content Ingestion
----------------------------------------
✅ Status: Successfully ingested 1 documents (1 chunks)
📊 document_ids: ['710bc20a-ddfa-4922-a9c2-c166f7e04f53']
📊 documents_ingested: 1

📝 Stored 1 document IDs from file content


In [11]:
# Method 3: URL Scraping
print("\n🌐 Method 3: URL Scraping")

# Example URLs to scrape (replace with actual URLs you want to scrape)
url_requests = [
    {
        "url": "https://docs.python.org/3/tutorial/introduction.html",
        "title": "Python Tutorial Introduction",
        "scraping_method": "auto"
    }
]

url_doc_ids = []

for url_request in url_requests:
    print(f"\n🔍 Scraping: {url_request['url']}")
    
    # Use async function with asyncio
    success, message, document_id = await document_service.ingest_document_from_url(
        url=url_request["url"],
        organization_id=ORGANIZATION_ID,
        title=url_request["title"],
        scraping_method=url_request["scraping_method"]
    )
    
    print_result(f"URL Scraping: {url_request['title']}", success, message, {
        "document_id": document_id,
        "url": url_request["url"],
        "scraping_method": url_request["scraping_method"]
    })
    
    if success and document_id:
        url_doc_ids.append(document_id)

print(f"\n📝 Stored {len(url_doc_ids)} document IDs from URL scraping")

# Combine all document IDs
all_document_ids = (direct_doc_ids or []) + (file_doc_ids or []) + url_doc_ids
print(f"\n📚 Total documents ingested: {len(all_document_ids)}")
print(f"Document IDs: {all_document_ids}")


🌐 Method 3: URL Scraping

🔍 Scraping: https://docs.python.org/3/tutorial/introduction.html


Calculating embeddings: 1it [00:01,  1.79s/it]



📋 URL Scraping: Python Tutorial Introduction
----------------------------------------
✅ Status: Successfully ingested document from URL: https://docs.python.org/3/tutorial/introduction.html
📊 document_id: 8f51e877-0741-46ff-a985-70074c1f8ba3
📊 url: https://docs.python.org/3/tutorial/introduction.html
📊 scraping_method: auto

📝 Stored 1 document IDs from URL scraping

📚 Total documents ingested: 4
Document IDs: ['df433591-c4ff-499b-84ea-4d9c087a9fc2', 'e2b78f2d-ec18-40a6-9ffd-9eaf861d1734', '710bc20a-ddfa-4922-a9c2-c166f7e04f53', '8f51e877-0741-46ff-a985-70074c1f8ba3']


In [12]:
# Verify organization and documents
updated_organization = document_service.get_or_create_organization(ORGANIZATION_ID)

print(f"📊 Organization Summary:")
print(f"   Name: {updated_organization.name}")
print(f"   Documents: {updated_organization.documents_count}")
print(f"   Created: {updated_organization.created_at}")

# Get organization stats
org_stats = document_service.get_organization_stats(ORGANIZATION_ID)
if "error" not in org_stats:
    print(f"\n📈 Organization Statistics:")
    print(f"   Total Documents: {org_stats['total_documents']}")
    print(f"   Total Chunks: {org_stats['total_chunks']}")
    print(f"   Total Content Length: {org_stats['total_content_length']:,} characters")
    print(f"   Average Chunks per Document: {org_stats['average_chunks_per_document']:.1f}")

📊 Organization Summary:
   Name: Example Organization
   Documents: 4
   Created: 2025-06-21 02:14:13.899903+00:00

📈 Organization Statistics:
   Total Documents: 4
   Total Chunks: 17
   Total Content Length: 25,377 characters
   Average Chunks per Document: 4.2


## 4. Campaign Creation

In [13]:
# Create a new campaign
campaign_request = CampaignCreateRequest(
    name="Python Learning Community Outreach 2024",
    description="Engage with Python learning communities to share knowledge and best practices",
    response_tone=ResponseTone.HELPFUL,
    max_responses_per_day=5
)

success, message, campaign = await campaign_service.create_campaign(
    organization_id=ORGANIZATION_ID,
    request=campaign_request
)

print_result("Campaign Creation", success, message)

# Store campaign for later use
campaign_id = None
if success and campaign:
    campaign_id = campaign.id
    print(f"\n🎯 Campaign ID: {campaign_id}")
    print(f"📝 Campaign Name: {campaign.name}")
    print(f"📊 Status: {campaign.status}")
    print(f"🎵 Tone: {campaign.response_tone}")
    print(f"📅 Created: {campaign.created_at}")
else:
    print("❌ Failed to create campaign")


📋 Campaign Creation
----------------------------------------
✅ Status: Campaign 'Python Learning Community Outreach 2024' created successfully

🎯 Campaign ID: ea91c59b-d77d-40b2-8f0f-ed9e883011ef
📝 Campaign Name: Python Learning Community Outreach 2024
📊 Status: CampaignStatus.CREATED
🎵 Tone: ResponseTone.HELPFUL
📅 Created: 2025-06-21 02:14:57.933579+00:00


## 5. Topic Discovery

First, we'll extract relevant topics from our documents.

In [14]:
if campaign_id and all_document_ids:
    print("🔍 Step 1: Discovering Topics from Documents")
    
    # Discover topics from selected documents
    topic_discovery_request = SubredditDiscoveryRequest(
        document_ids=all_document_ids[:3]  # Use first 3 documents for topic discovery
    )
    
    success, message, topic_data = await campaign_service.discover_topics(
        campaign_id=campaign_id,
        request=topic_discovery_request
    )
    
    print_result("Topic Discovery", success, message, topic_data)
    
    # Extract topics for next step
    discovered_topics = []
    if success and topic_data and "topics" in topic_data:
        discovered_topics = topic_data["topics"]
        print(f"\n📋 Discovered Topics:")
        for i, topic in enumerate(discovered_topics, 1):
            print(f"   {i}. {topic}")
    
    # Check campaign status
    success_status, message_status, updated_campaign = await campaign_service.get_campaign(campaign_id)
    if success_status and updated_campaign:
        print(f"\n📊 Campaign Status: {updated_campaign.status}")
        print(f"📄 Documents Selected: {len(updated_campaign.selected_document_ids)}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or document IDs")
    discovered_topics = []

🔍 Step 1: Discovering Topics from Documents

📋 Topic Discovery
----------------------------------------
✅ Status: Extracted 10 topics from 3 documents
📊 topics: ['Python best practices', 'PEP 8 style guide', 'Machine learning', 'Supervised learning', 'Unsupervised learning', 'Reinforcement learning', 'Web development with Python', 'FastAPI', 'Python testing', 'Error handling in Python']
📊 selected_document_ids: ['df433591-c4ff-499b-84ea-4d9c087a9fc2', 'e2b78f2d-ec18-40a6-9ffd-9eaf861d1734', '710bc20a-ddfa-4922-a9c2-c166f7e04f53']
📊 total_topics: 10

📋 Discovered Topics:
   1. Python best practices
   2. PEP 8 style guide
   3. Machine learning
   4. Supervised learning
   5. Unsupervised learning
   6. Reinforcement learning
   7. Web development with Python
   8. FastAPI
   9. Python testing
   10. Error handling in Python

📊 Campaign Status: CampaignStatus.DOCUMENTS_UPLOADED
📄 Documents Selected: 3


## 6. Subreddit Discovery

Now we'll use the discovered topics to find relevant subreddits.

In [15]:
if campaign_id and discovered_topics:
    print("🎯 Step 2: Discovering Subreddits from Topics")
    
    # Discover subreddits based on topics
    subreddit_discovery_request = SubredditDiscoveryByTopicsRequest(
        topics=discovered_topics
    )
    
    success, message, subreddit_data = await campaign_service.discover_subreddits(
        campaign_id=campaign_id,
        request=subreddit_discovery_request
    )
    
    print_result("Subreddit Discovery", success, message, subreddit_data)
    
    # Extract subreddits for next step
    target_subreddits = []
    if success and subreddit_data and "subreddits" in subreddit_data:
        target_subreddits = subreddit_data["subreddits"]
        print(f"\n🎯 Target Subreddits:")
        for i, subreddit in enumerate(target_subreddits, 1):
            print(f"   {i}. r/{subreddit}")
    
    # Check updated campaign status
    success_status, message_status, updated_campaign = await campaign_service.get_campaign(campaign_id)
    if success_status and updated_campaign:
        print(f"\n📊 Campaign Status: {updated_campaign.status}")
        print(f"🎯 Subreddits Found: {len(updated_campaign.target_subreddits)}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or topics")
    target_subreddits = []

🎯 Step 2: Discovering Subreddits from Topics


Error: received 403 HTTP response. Retrying in 2.00 seconds...
Error: received 403 HTTP response. Retrying in 4.00 seconds...
Error: received 403 HTTP response. Retrying in 8.00 seconds...
Error: received 403 HTTP response. Retrying in 2.00 seconds...
Error: received 403 HTTP response. Retrying in 2.00 seconds...
Error: received 403 HTTP response. Retrying in 4.00 seconds...
Error: received 403 HTTP response. Retrying in 4.00 seconds...
Max retries reached: received 403 HTTP response
Error getting info for r/thisisthewayitwillbe: received 403 HTTP response
Error getting details for r/thisisthewayitwillbe: received 403 HTTP response
Error: received 403 HTTP response. Retrying in 8.00 seconds...
Error: received 403 HTTP response. Retrying in 8.00 seconds...
Max retries reached: received 403 HTTP response
Error getting info for r/thisisthewayitwillbe: received 403 HTTP response
Error getting details for r/thisisthewayitwillbe: received 403 HTTP response
Max retries reached: received 403 H


📋 Subreddit Discovery
----------------------------------------
✅ Status: Discovered 10 relevant subreddits
📊 subreddits: ['Python', 'learnpython', 'pythontips', 'MachineLearning', 'learnmachinelearning', 'datascience', 'artificial', 'ArtificialInteligence', 'deeplearning', 'FastAPI']
📊 topics: ['Python best practices', 'PEP 8 style guide', 'Machine learning', 'Supervised learning', 'Unsupervised learning', 'Reinforcement learning', 'Web development with Python', 'FastAPI', 'Python testing', 'Error handling in Python']
📊 total_found: 10

🎯 Target Subreddits:
   1. r/Python
   2. r/learnpython
   3. r/pythontips
   4. r/MachineLearning
   5. r/learnmachinelearning
   6. r/datascience
   7. r/artificial
   8. r/ArtificialInteligence
   9. r/deeplearning
   10. r/FastAPI

📊 Campaign Status: CampaignStatus.SUBREDDITS_DISCOVERED
🎯 Subreddits Found: 10


## 7. Post Discovery

In [ ]:
if campaign_id and target_subreddits:
    print("📝 Step 3: Discovering Relevant Posts")
    
    # Discover posts in target subreddits
    post_discovery_request = PostDiscoveryRequest(
        subreddits=target_subreddits[:3],  # Limit to first 3 subreddits for demo
        max_posts_per_subreddit=5,
        time_filter="week",
        reddit_credentials=REDDIT_CREDENTIALS
    )
    
    success, message, posts_data = await campaign_service.discover_posts(
        campaign_id=campaign_id,
        request=post_discovery_request
    )
    
    print_result("Post Discovery", success, message, posts_data)
    
    # Extract post information
    target_posts = []
    if success and posts_data and "posts" in posts_data:
        target_posts = posts_data["posts"]
        print(f"\n📝 Found {len(target_posts)} relevant posts:")
        for i, post in enumerate(target_posts[:5], 1):  # Show first 5
            print(f"   {i}. r/{post['subreddit']}: {post['title'][:60]}...")
            print(f"      Relevance: {post['relevance_score']:.2f} - {post['relevance_reason']}")
    
    # Check campaign status
    success_status, message_status, updated_campaign = await campaign_service.get_campaign(campaign_id)
    if success_status and updated_campaign:
        print(f"\n📊 Campaign Status: {updated_campaign.status}")
        print(f"📝 Posts Found: {len(updated_campaign.target_posts)}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or subreddits")
    target_posts = []

## 8. Response Generation

In [ ]:
if campaign_id and target_posts:
    print("💬 Step 4: Generating Responses")
    
    # Get post IDs for response generation
    post_ids = [post["id"] for post in target_posts[:3]]  # Limit to first 3 posts
    
    response_generation_request = ResponseGenerationRequest(
        target_post_ids=post_ids,
        tone=ResponseTone.HELPFUL
    )
    
    success, message, generation_data = await campaign_service.generate_responses(
        campaign_id=campaign_id,
        request=response_generation_request
    )
    
    print_result("Response Generation", success, message, generation_data)
    
    # Show generated responses
    planned_responses = []
    if success and generation_data and "responses" in generation_data:
        planned_responses = generation_data["responses"]
        print(f"\n💬 Generated {len(planned_responses)} responses:")
        for i, response in enumerate(planned_responses, 1):
            print(f"\n   Response {i}:")
            print(f"   Target Post: {response['target_post_id']}")
            print(f"   Confidence: {response['confidence_score']:.2f}")
            print(f"   Content Preview: {response['response_content'][:100]}...")
    
    # Check campaign status
    success_status, message_status, updated_campaign = await campaign_service.get_campaign(campaign_id)
    if success_status and updated_campaign:
        print(f"\n📊 Campaign Status: {updated_campaign.status}")
        print(f"💬 Responses Planned: {len(updated_campaign.planned_responses)}")
        
else:
    print("❌ Cannot proceed: Missing campaign ID or posts")
    planned_responses = []

## 9. Response Execution (Optional)

⚠️ **WARNING**: This step will actually post to Reddit if `ACTUALLY_POST_TO_REDDIT = True`

In [ ]:
if campaign_id and planned_responses:
    if ACTUALLY_POST_TO_REDDIT:
        print("🚀 Step 5: Executing Responses (POSTING TO REDDIT)")
        
        # Get response IDs for execution
        response_ids = [response["id"] for response in planned_responses[:2]]  # Limit to first 2
        
        execution_request = ResponseExecutionRequest(
            planned_response_ids=response_ids,
            reddit_credentials=REDDIT_CREDENTIALS
        )
        
        success, message, execution_data = await campaign_service.execute_responses(
            campaign_id=campaign_id,
            request=execution_request
        )
        
        print_result("Response Execution", success, message, execution_data)
        
        # Show execution results
        if success and execution_data and "posted_responses" in execution_data:
            posted_responses = execution_data["posted_responses"]
            print(f"\n🚀 Execution Results:")
            for i, response in enumerate(posted_responses, 1):
                status = "✅ Success" if response["posting_successful"] else "❌ Failed"
                print(f"   Response {i}: {status}")
                if response["posting_successful"]:
                    print(f"   Reddit URL: https://reddit.com{response['reddit_permalink']}")
                else:
                    print(f"   Error: {response.get('error_message', 'Unknown error')}")
        
        # Final campaign status
        success_status, message_status, final_campaign = await campaign_service.get_campaign(campaign_id)
        if success_status and final_campaign:
            print(f"\n📊 Final Campaign Status: {final_campaign.status}")
            print(f"🚀 Responses Posted: {len(final_campaign.posted_responses)}")
            successful_posts = len([r for r in final_campaign.posted_responses.values() if r.posting_successful])
            failed_posts = len([r for r in final_campaign.posted_responses.values() if not r.posting_successful])
            print(f"✅ Successful Posts: {successful_posts}")
            print(f"❌ Failed Posts: {failed_posts}")
    else:
        print("⚠️  Step 5: Response Execution SKIPPED (Safe Mode)")
        print("\n🛡️  Reddit posting is disabled for safety.")
        print("   To enable posting, set ACTUALLY_POST_TO_REDDIT = True")
        print("   and provide valid Reddit credentials.")
        
        print(f"\n📋 Would have posted {len(planned_responses)} responses:")
        for i, response in enumerate(planned_responses, 1):
            print(f"   {i}. Response with confidence {response['confidence_score']:.2f}")
            print(f"      Content: {response['response_content'][:80]}...")
else:
    print("❌ Cannot proceed: Missing campaign ID or planned responses")

## 10. Analytics & Reporting

In [ ]:
print("📊 Step 6: Analytics & Reporting")

# Get organization quick stats
quick_stats = analytics_service.get_quick_stats(ORGANIZATION_ID)
print_result("Organization Quick Stats", "error" not in quick_stats, "Quick stats retrieved", quick_stats)

# Get organization performance report
performance_report = analytics_service.get_organization_performance_report(ORGANIZATION_ID)
print_result("Organization Performance Report", "error" not in performance_report, "Performance report generated", {
    "report_type": performance_report.get("report_type"),
    "insights_count": len(performance_report.get("performance_insights", [])),
    "campaign_stats": performance_report.get("campaign_stats", {})
})

if campaign_id:
    # Get campaign engagement report
    engagement_report = analytics_service.get_campaign_engagement_report(campaign_id)
    print_result("Campaign Engagement Report", "error" not in engagement_report, "Engagement report generated", {
        "campaign_name": engagement_report.get("campaign_name"),
        "status": engagement_report.get("status"),
        "basic_stats": engagement_report.get("basic_stats", {})
    })

# Get platform overview
platform_overview = analytics_service.get_overall_platform_metrics()
print_result("Platform Overview", "error" not in platform_overview, "Platform overview generated", {
    "total_campaigns": platform_overview.get("campaign_stats", {}).get("total_campaigns", 0),
    "total_organizations": platform_overview.get("campaign_stats", {}).get("total_organizations", 0),
    "insights_count": len(platform_overview.get("platform_insights", []))
})

# Get subreddit effectiveness report
subreddit_effectiveness = analytics_service.get_subreddit_effectiveness_report(ORGANIZATION_ID)
print_result("Subreddit Effectiveness", "error" not in subreddit_effectiveness, "Subreddit effectiveness analyzed", {
    "total_subreddits": subreddit_effectiveness.get("total_subreddits_analyzed", 0),
    "recommendations_count": len(subreddit_effectiveness.get("recommendations", []))
})

## 11. Additional Operations

In [ ]:
# Query documents
print("🔍 Document Query Example")

query = DocumentQuery(
    query="machine learning algorithms",
    organization_id=ORGANIZATION_ID,
    method="semantic",
    top_k=3
)

query_response = document_service.query_documents(query)

print_result("Document Query", True, f"Found {query_response.total_results} documents", {
    "query": query_response.query,
    "method": query_response.method,
    "processing_time_ms": query_response.processing_time_ms,
    "total_results": query_response.total_results
})

if query_response.documents:
    print(f"\n📄 Found {len(query_response.documents)} relevant documents:")
    for i, doc in enumerate(query_response.documents, 1):
        print(f"   {i}. {doc.title} (Score: {doc.score:.3f})")
        print(f"      Content: {doc.content[:100]}...")

In [ ]:
# List all campaigns for the organization
print("📋 List All Campaigns")

success, message, campaigns = await campaign_service.list_campaigns(ORGANIZATION_ID)

print_result("Organization Campaigns", success, message, {
    "campaigns_count": len(campaigns) if campaigns else 0
})

if success and campaigns:
    print(f"\n📊 Found {len(campaigns)} campaigns:")
    for i, campaign in enumerate(campaigns, 1):
        print(f"   {i}. {campaign.name} ({campaign.status})")
        print(f"      Created: {campaign.created_at}")
        print(f"      ID: {campaign.id}")

In [ ]:
# Test subreddit search
print("🔍 Subreddit Search Example")

success, message, results = await reddit_service.search_subreddits("python programming", limit=5)

print_result("Subreddit Search", success, message, {
    "results_count": len(results) if results else 0
})

if success and results:
    print(f"\n🎯 Found {len(results)} subreddits:")
    for i, subreddit in enumerate(results, 1):
        print(f"   {i}. r/{subreddit['name']} ({subreddit['subscribers']:,} subscribers)")
        print(f"      Description: {subreddit['description'][:80]}...")

## 12. Cleanup & Summary

In [ ]:
# Cleanup resources
print("🧹 Cleaning up resources...")

# Close Reddit client connection
await reddit_service.cleanup()
await campaign_service.cleanup()

print("✅ Resources cleaned up successfully")

# Summary
print("\n📋 Workflow Summary")
print("=" * 50)

print(f"🏢 Organization: {ORGANIZATION_NAME} ({ORGANIZATION_ID})")
print(f"📚 Documents Ingested: {len(all_document_ids)}")
print(f"   - Direct Content: {len(direct_doc_ids) if direct_doc_ids else 0}")
print(f"   - File Upload: {len(file_doc_ids) if file_doc_ids else 0}")
print(f"   - URL Scraping: {len(url_doc_ids)}")

if campaign_id:
    print(f"\n🎯 Campaign: {campaign_id}")
    print(f"🔍 Topics Discovered: {len(discovered_topics)}")
    print(f"🎯 Subreddits Found: {len(target_subreddits)}")
    print(f"📝 Posts Analyzed: {len(target_posts)}")
    print(f"💬 Responses Generated: {len(planned_responses)}")
    
    if ACTUALLY_POST_TO_REDDIT:
        print(f"🚀 Responses Posted: Executed")
    else:
        print(f"🛡️  Responses Posted: Skipped (Safe Mode)")

print(f"\n⏰ Completed at: {datetime.now()}")
print(f"✅ Workflow completed successfully!")

# Display key IDs for reference
print("\n🔑 Key IDs for Reference:")
print(f"   Organization ID: {ORGANIZATION_ID}")
if campaign_id:
    print(f"   Campaign ID: {campaign_id}")
if all_document_ids:
    print(f"   Document IDs: {all_document_ids}")

print("\n" + "=" * 50)
print("🎉 Reddit Marketing AI Agent Workflow Complete!")
print("\n💡 This workflow used direct function calls instead of API endpoints.")
print("   This demonstrates how to use the services programmatically.")

## Conclusion

This notebook has demonstrated the complete workflow of the Reddit Marketing AI Agent using **direct function calls** instead of API endpoints:

1. ✅ **Setup & Configuration** - Initialized all services and managers directly
2. ✅ **Organization Setup** - Used DocumentService to create/verify organization
3. ✅ **Document Ingestion** - Demonstrated all three ingestion methods via DocumentService
4. ✅ **Campaign Creation** - Created campaign using CampaignService
5. ✅ **Topic Discovery** - Extracted topics using CampaignService.discover_topics()
6. ✅ **Subreddit Discovery** - Found subreddits using CampaignService.discover_subreddits()
7. ✅ **Post Discovery** - Identified posts using CampaignService.discover_posts()
8. ✅ **Response Generation** - Generated responses using CampaignService.generate_responses()
9. ✅ **Response Execution** - Demonstrated posting workflow (with safety controls)
10. ✅ **Analytics & Reporting** - Generated reports using AnalyticsService

### Key Differences from API Approach:
- **Direct Service Access**: Called service methods directly instead of HTTP endpoints
- **Better Performance**: No HTTP overhead, faster execution
- **Type Safety**: Full Python type checking and IDE support
- **Easier Debugging**: Direct access to service internals
- **Resource Management**: Explicit cleanup of connections and resources

### Service Architecture Demonstrated:
- **DocumentService**: Document ingestion, querying, and management
- **CampaignService**: Complete campaign workflow orchestration
- **RedditService**: Reddit API interactions and subreddit operations
- **LLMService**: AI-powered topic extraction and response generation
- **AnalyticsService**: Comprehensive reporting and analytics
- **WebScraperService**: URL content scraping capabilities

### Next Steps:
1. **Integrate into Applications**: Use these service patterns in your own applications
2. **Customize Workflows**: Modify the services to fit your specific needs
3. **Add Error Handling**: Implement robust error handling for production use
4. **Scale Operations**: Use these patterns for batch processing and automation
5. **Monitor Performance**: Add logging and monitoring to track service performance

The Reddit Marketing AI Agent services are now ready for programmatic integration! 🚀